In [5]:
import requests
import asyncio
from playwright.async_api import async_playwright
from urllib.parse import urljoin
from pyquery import PyQuery as pq
import time
import json
import re
from motor.motor_asyncio import AsyncIOMotorClient
import logging
from pprint import pprint
import sys

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

def scrape_page(url):
    logging.info('scraping %s', url)
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        logging.info('get invalid status code %s while scraping %s', response.status_code, url)
    except requests.RequestException:
        logging.error('error occurred while scraping %s', url, exc_info=True)

In [21]:
Whitelist_API = 'https://wapi.proxy.linkudp.com/api/white_list?neek=427888&appkey=30fe435f76fd09d1e5a6f001abd94361'
white_list_api = Whitelist_API
list_json = scrape_page(white_list_api)
pprint(list_json)

2024-03-23 12:26:34,786 - INFO: scraping https://wapi.proxy.linkudp.com/api/white_list?neek=427888&appkey=30fe435f76fd09d1e5a6f001abd94361


{'code': 0,
 'data': {'day_white_balance': 30,
          'day_white_limit': 30,
          'lists': [{'mark_ip': '113.87.80.88', 'updated_at': 1711167260},
                    {'mark_ip': '113.87.80.66', 'updated_at': 1711167246},
                    {'mark_ip': '113.87.80.33', 'updated_at': 1711167227},
                    {'mark_ip': '113.87.80.99', 'updated_at': 1711166831},
                    {'mark_ip': '120.25.240.33', 'updated_at': 1711166122}],
          'mark': {'day_white_balance': '单日剩余修改白名单次数',
                   'day_white_limit': '单日修改用户白名单总数',
                   'lists': '白名单列表',
                   'white': '白名单总数'},
          'white': 5},
 'msg': 'ok'}


In [22]:
your_ip = '113.87.80.21'
Whiteset_API = 'https://wapi.http.linkudp.com/index/index/save_white?neek=427888&appkey=30fe435f76fd09d1e5a6f001abd94361&white='

if list_json['code'] == 0:
    lists = list_json['data']['lists']
    white_list = []
    for list in lists:
        white_list.append(list['mark_ip'])
    if your_ip in white_list:
        logging.info('Your ip is exit')
    else:
        url = Whiteset_API + your_ip
        result = scrape_page(url)
        if result['code'] == 0:
            logging.info('Save ip to white list successed')
        else:
            logging.error('Please check white list set error code:', result['code'])
else:
    logging.error('Withe list api is error %s', white_list_api)

2024-03-23 12:27:13,242 - INFO: scraping https://wapi.http.linkudp.com/index/index/save_white?neek=427888&appkey=30fe435f76fd09d1e5a6f001abd94361&white=113.87.80.21
2024-03-23 12:27:13,577 - INFO: Save ip to white list successed


In [23]:
Direct_IP = 'http://webapi.http.zhimacangku.com/getip?neek=860483b89fc4d67d&num=10&type=2&pro=&city=0&yys=0&port=11&time=4&ts=1&ys=1&cs=1&lb=1&sb=0&pb=4&mr=1&regions='
ip_api = Direct_IP # 98
data = scrape_page(ip_api)
pprint(data)

2024-03-23 12:27:23,014 - INFO: scraping http://webapi.http.zhimacangku.com/getip?neek=860483b89fc4d67d&num=10&type=2&pro=&city=0&yys=0&port=11&time=4&ts=1&ys=1&cs=1&lb=1&sb=0&pb=4&mr=1&regions=


{'code': 0,
 'data': [{'city': '浙江省杭州市萧山区',
           'expire_time': '2024-03-23 12:31:53',
           'ip': '183.128.97.182',
           'isp': '电信',
           'port': 4215},
          {'city': '四川省绵阳市',
           'expire_time': '2024-03-23 12:31:53',
           'ip': '139.200.75.95',
           'isp': '电信',
           'port': 4283},
          {'city': '湖北省黄冈市',
           'expire_time': '2024-03-23 12:31:53',
           'ip': '119.102.44.45',
           'isp': '电信',
           'port': 4213},
          {'city': '河北省秦皇岛市',
           'expire_time': '2024-03-23 12:31:53',
           'ip': '123.181.235.235',
           'isp': '电信',
           'port': 4227},
          {'city': '福建省漳州市',
           'expire_time': '2024-03-23 12:31:53',
           'ip': '59.60.140.194',
           'isp': '电信',
           'port': 4215},
          {'city': '浙江省舟山市',
           'expire_time': '2024-03-23 12:31:53',
           'ip': '123.96.168.2',
           'isp': '电信',
           'port': 4236},
          

In [37]:
if data['code'] == 0:
    servers = []
    for item in data['data']:
        ip = item['ip']
        port = item['port']
        server = ip + ':' + str(port)
        logging.info(server)
    return servers
else:
    

2024-03-23 12:45:20,382 - INFO: 183.128.97.182:4215
2024-03-23 12:45:20,382 - INFO: 139.200.75.95:4283
2024-03-23 12:45:20,383 - INFO: 119.102.44.45:4213
2024-03-23 12:45:20,383 - INFO: 123.181.235.235:4227
2024-03-23 12:45:20,384 - INFO: 59.60.140.194:4215
2024-03-23 12:45:20,385 - INFO: 123.96.168.2:4236
2024-03-23 12:45:20,385 - INFO: 116.208.199.211:4213
2024-03-23 12:45:20,386 - INFO: 182.34.150.158:4214
2024-03-23 12:45:20,387 - INFO: 183.141.7.81:4214
2024-03-23 12:45:20,387 - INFO: 60.185.204.158:4213


In [30]:
type(data['data'][0])

dict

In [5]:
proxy = ['183.128.97.182:4215',
         '139.200.75.95:4283',
         '119.102.44.45:4213',
         '123.181.235.235:4227',
         '59.60.140.194:4215',
         '123.96.168.2:4236',
         '116.208.199.211:4213',
         '182.34.150.158:4214',
         '183.141.7.81:4214',
         '60.185.204.158:4213'
         ]

In [22]:
PROXY_SERVER = ''
PROXY_LIST = proxy

def set_proxy():
    global PROXY_LIST
    global proxy
    if len(PROXY_LIST) == 0:
        PROXY_LIST = proxy
        logging.info('list is: %s', PROXY_LIST)
    PROXY_SERVER = PROXY_LIST[-1]
    logging.info('server is : %s', PROXY_SERVER)
    PROXY_LIST = PROXY_LIST[0:-1]

In [23]:
for _ in range(20):
    set_proxy()

2024-03-26 17:02:38,813 - INFO: server is : 60.185.204.158:4213
2024-03-26 17:02:38,815 - INFO: server is : 183.141.7.81:4214
2024-03-26 17:02:38,815 - INFO: server is : 182.34.150.158:4214
2024-03-26 17:02:38,816 - INFO: server is : 116.208.199.211:4213
2024-03-26 17:02:38,817 - INFO: server is : 123.96.168.2:4236
2024-03-26 17:02:38,818 - INFO: server is : 59.60.140.194:4215
2024-03-26 17:02:38,819 - INFO: server is : 123.181.235.235:4227
2024-03-26 17:02:38,820 - INFO: server is : 119.102.44.45:4213
2024-03-26 17:02:38,821 - INFO: server is : 139.200.75.95:4283
2024-03-26 17:02:38,821 - INFO: server is : 183.128.97.182:4215
2024-03-26 17:02:38,823 - INFO: list is: ['183.128.97.182:4215', '139.200.75.95:4283', '119.102.44.45:4213', '123.181.235.235:4227', '59.60.140.194:4215', '123.96.168.2:4236', '116.208.199.211:4213', '182.34.150.158:4214', '183.141.7.81:4214', '60.185.204.158:4213']
2024-03-26 17:02:38,824 - INFO: server is : 60.185.204.158:4213
2024-03-26 17:02:38,824 - INFO: se

In [24]:
response = '''
{
  "args": {}, 
  "headers": {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
    "Accept-Encoding": "gzip, deflate, br, zstd", 
    "Accept-Language": "zh-CN,zh;q=0.9", 
    "Cache-Control": "max-age=0", 
    "Host": "httpbin.org", 
    "Sec-Ch-Ua": "\"Chromium\";v=\"122\", \"Not(A:Brand\";v=\"24\", \"Google Chrome\";v=\"122\"", 
    "Sec-Ch-Ua-Mobile": "?0", 
    "Sec-Ch-Ua-Platform": "\"Windows\"", 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "none", 
    "Sec-Fetch-User": "?1", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-66039326-1bf15970515e263a00f985c9"
  }, 
  "origin": "223.104.68.50", 
  "url": "https://httpbin.org/get"
}
'''

In [25]:
print(response)


{
  "args": {}, 
  "headers": {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
    "Accept-Encoding": "gzip, deflate, br, zstd", 
    "Accept-Language": "zh-CN,zh;q=0.9", 
    "Cache-Control": "max-age=0", 
    "Host": "httpbin.org", 
    "Sec-Ch-Ua": ""Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"", 
    "Sec-Ch-Ua-Mobile": "?0", 
    "Sec-Ch-Ua-Platform": ""Windows"", 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "none", 
    "Sec-Fetch-User": "?1", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36", 
    "X-Amzn-Trace-Id": "Root=1-66039326-1bf15970515e263a00f985c9"
  }, 
  "origin": "223.104.68.50", 
  "url": "https://httpbin.org/get"
}



In [31]:
re.search(r'"origin": "(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', response).group(1)

'223.104.68.50'

In [4]:
USERNAME = 'admin'
PASSWORD = 'admin'
ACCOUNT = [
    ['herozhou1', 'asdfgh'],['herozhou2', 'asdfgh'],['herozhou3', 'asdfgh'],['herozhou4', 'asdfgh'],['herozhou5', 'asdfgh'],
    ['herozhou6', 'asdfgh'],['herozhou7', 'asdfgh'],['herozhou8', 'asdfgh'],['herozhou9', 'asdfgh'],['herozhou10', 'asdfgh'],
    ]

In [3]:
def set_account():
    global USERNAME
    global PASSWORD
    global ACCOUNT
    if len(ACCOUNT) == 0:
        sys.exit(1)
    USERNAME = ACCOUNT[-1][0]
    PASSWORD = ACCOUNT[-1][1]
    ACCOUNT = ACCOUNT[0:-1]

In [6]:
set_account()
print(USERNAME)
print(PASSWORD)

herozhou10
asdfgh


In [2]:
book_url.get('name')

'feitu'

In [4]:
'''
程序作用: 爬取指定页面，返回页面文本
参数: url (爬取页面地址)
返回值: response.text (页面文本)
'''
def scrape_page(url):   # 定义函数为scrape_page
    # url = 'https://ssr1.scrape.center/page/1' 
    logging.info('scraping %s', url)  # 以info级别的日志信息输出这个“url"网址  
    # xxx(xxx, xxx); xxx(xxx) scraping https://ssr1.scrape.center/page/1
    try: # 异常处理 try 和 except
        response = requests.get(url)    # 爬取 请求网址响应
        if response.status_code == 200: # 状态码为200时返回页面文本 
            return response.text        # 返回页面文本 
        logging.info('get invalid status code %s while scraping %s', response.status_code, url)
        # 以info级别的日志信息输出 “获取无效状态代码  或者 解析失败   url= https://ssr1.scrape.center/page/1”
    except requests.RequestException:    # 如果请求异常 
        logging.error('error occurred while scraping %s', url, exc_info=True) 
        # 以error级别的日志信息输出 “请求异常  url= https://ssr1.scrape.center/page/1”

In [5]:
URL = 'http://www.qzxs.cc/xiaoshuo/rbd2vo1/chapter/32g5vo1.html'

In [6]:
print(scrape_page(URL))

2024-02-17 23:06:02,550 - INFO: scraping http://www.qzxs.cc/xiaoshuo/rbd2vo1/chapter/32g5vo1.html


<!DOCTYPE html><html lang="zh-cmn-Hans" class="theme5"><head><meta charset="UTF-8"><title>第四百六七节 幽灵-废土(黑天魔神)-棋子小说</title><meta name="keywords" content="第四百六七节 幽灵,废土,黑天魔神,棋子小说"><meta name="description" content="第四百六七节 幽灵是黑天魔神所著奇幻小说《废土》的最新章节，棋子小说提供无弹窗阅读！"><link href="http://www.qzxs.cc/xiaoshuo/rbd2vo1/chapter/32g5vo1.html" rel="canonical"><meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1, user-scalable=no, viewport-fit=cover"><meta name="renderer" content="webkit|ie-comp|ie-stand"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta http-equiv="Cache-Control" content="no-transform"><meta http-equiv="Cache-Control" content="no-siteapp"><meta name="applicable-device" content="pc,mobile"><link rel="stylesheet" type="text/css" href="/static/biquge2023/book.css?v=20231124"><script>var classVal="theme5";if(window.localStorage.getItem("bgcolor")){document.getElementsByTagName("html")[0].setAttribute("class",window.localStorage.g

In [5]:
doc = pq(filename='32g5vo1.html')

In [7]:
title = doc('.read .booktitle h1').text()
print(title)

第四百六七节 幽灵


In [8]:
content = doc('.read .content p').text()
print(content)

黑天魔神提示您：看后求收藏（棋子小说qzxs.cc），接着再看更方便。 走廊，非常幽深。 配合几分钟前刚刚离开的电梯，还有表示楼层的“一21，字样，即便是林翔，也有种从地面直接坠入地狱深处的错觉。 在一道六棱形钢制大门前，雷契尔将军停下脚步，从贴身衣袋里取出一张磁卡，用力刷过旁边墙壁上的电子感应器。随即，从棱形门壁的背后，传来齿轮转动的清晰声响。 “这里并不设防。” 将军深深地吸了口气，说：“你可能会觉得奇怪，为什么不适用保密级别更高的虹膜探测或者个人指纹密码？但你很快就能知道，这些所谓的屏障，在那个人面前根本没有丝毫效果。即便没有通行卡，你也很容易就能进入。我并不是刻意制止或者故意恐吓，那个人。”。”他已经超乎正常想象的范围。或许，只有你和他之间，才有真正能够谈论的话题。但是不管怎么样，我希望你能够记住之前说过的那些话。这个世界已经太过混乱，我不希望看到战争，有很多人。”。”不应该默默无闻的死去。” 林翔黑色的瞳孔焦点落在了雷契尔身上。刹那间，将军只觉得自己的身体从内到外都冒出森森寒意，似乎已变成完全透明。在这种无所遁形的目光下，所有的秘密都无从隐藏。 “你和我所见过的一些骷髅骑士不同。至少。”。”你仍然保持着某种信念。” 林翔平静地说：、‘向城外的军队请降吧！你并非战败，只是因为责任。” 。”。” 全封闭的棱形走庇，很宽敞。地面与头顶都装设着淡蓝色的冷光灯，黑暗被彻底驱逐，却丝毫感觉不到温暖。 尽头，是一道同样形状的感应门。不需要触摸，它已经自动分朝两边，让出足够通行的过道。走进其中，林翔却如同被电击一样牢牢定住，站在原地，久久未能移开脚步。 这是一个非常宽大的房间，面积至少超过两千平米。沿着墙壁，摆放着各种复杂的电子仪器，还有一张张堆放器材的工台。柔和的白色灯光从头顶洒落下来，足够看清楚其中的每一个角落。 一个身穿白色家居便服的青年，正对着林翔，坐在房间南角的椅子上，聚精会神看着一本平放在桌面上的书。 他的外表年龄最多只有二十岁。眼睛是欧裔人种特有的淡蓝色，长长的金色头发，在脑后用黑色缎带系成马尾。皮肤很白，似乎长时间没有接触过阳光。在这种苍白肤色的衬托下，嘴唇上的淡红也显得越发鲜艳。尤其是从林翔站立的角度望去，感觉。”。”就像是传说中刚刚饮过人血的吸血鬼。 目光与这名男子接触的一刹那，林翔忽然产生出一种极其微妙，也非常奇怪的感觉。 自己。”。”似乎认识他。 

In [9]:
with open('novle.txt', 'w', encoding='utf-8') as f:
    f.write(content)

In [2]:
doc = pq(filename='book_detail.html')

In [4]:
doc('.comments').text()

'你会重新爱这个曾是你自己的陌生人。\n喜欢他以诗意和文学的形式解读武侠小说。他会用塞林格的短篇串联程灵素的悲剧，也会用博尔赫斯的哲学观来形容古龙笔下的英雄，还以萨特和萨冈来形容自己的所感所悟。他的文字充满才华，诗句也穿插妙用，很像以前读书时会仰慕的文科才子。\n大学时读过，现在重读，缅怀大学岁月。2020年读的第一本书。\n江湖中人\n品读武侠，北大才子读武侠和我们普通人就是不一样，我们看剧情热闹，人家看的是人生人事\n多年前听闻此书，一直没买到。作者对古金的人物，兵器，每个人的个性真的是如数家珍，借由这些引出自己的人生观及道路走向。书中看到很多北大才子们的诗作，对青春江湖的缅怀，是另一得。\n除却斯人海子的另外一个北大才子江湖。'

In [2]:
text1 = '13.80元'
text2 = '13'
text3 = '13元'
text4 = 'CNY 30.00'
text5 = '元'

In [3]:
result1 = re.search(r'\d+(\.\d*)?', text1).group()
print(result1)

13.80


In [14]:
result2 = re.search(r'\d*(\.\d*)?', text2).group()
print(result2)

13


In [15]:
result3 = re.search(r'\d*(\.\d*)?', text3).group()
print(result3)

13


In [16]:
result4 = re.search(r'\d+(\.\d*)?', text4).group()
print(result4)

30.00


In [7]:
result4 = re.search(r'\d*\.\d*', text4).group()
print(result4)

30.00


In [15]:
re.search(r'\d+', text4).group()

'30'

In [4]:
re.search(r'\d+(\.\d*)?', text1)

<re.Match object; span=(0, 5), match='13.80'>

In [7]:
print(type(re.search(r'\d+(\.\d*)?', text1)))

<class 're.Match'>


In [8]:
float(re.search(r'\d+(\.\d*)?', text1).group())

13.8